In [ ]:
import os, re, time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import StaleElementReferenceException

download_dir = r"C:\Users\rando\Downloads" 
driver = webdriver.Chrome()
wait = WebDriverWait(driver, 20)

driver.get("https://vahan.parivahan.gov.in/vahan4dashboard/vahan/vahan/view/reportview.xhtml")

# --- CSS selectors for dropdown items ---
state_items_css  = "#j_idt43_items li"
rto_items_css    = "#selectedRto_items li"
yaxis_items_css  = "#yaxisVar_items li"
xaxis_items_css  = "#xaxisVar_items li"
year_items_css   = "#selectedYear_items li"

# --- Helper for safe clicks ---
def safe_click(locator, retries=3, delay=1):
    for attempt in range(retries):
        try:
            el = wait.until(EC.element_to_be_clickable(locator))
            driver.execute_script("arguments[0].click();", el)
            return
        except StaleElementReferenceException:
            print("Stale element, retrying:", locator)
            time.sleep(delay)
    raise Exception(f"Could not click {locator} after {retries} retries")


# --- Select Month ---
def select_month(month_label, retries=3):
    for attempt in range(retries):
        try:
            # Always re-open the dropdown to get a fresh DOM
            safe_click((By.ID, "groupingTable:selectMonth_label"))
            time.sleep(0.4)

            # Re-locate the overlay items each time
            month_items = wait.until(
                EC.presence_of_all_elements_located((By.CSS_SELECTOR, "#groupingTable\\:selectMonth_items li"))
            )

            # Find the desired month by text
            for m in month_items:
                if m.text.strip() == month_label:
                    driver.execute_script("arguments[0].click();", m)
                    return
        except StaleElementReferenceException:
            print(f"Stale element while selecting {month_label}, retrying...")
            time.sleep(0.5)
    raise Exception(f"Could not select month {month_label} after {retries} retries")


all_dataframes = []

# --- Open States dropdown once to get list ---
safe_click((By.ID, "j_idt43_label")) #"j_idt38_label"
time.sleep(0.5)
states = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, state_items_css)))

for i in range(34, len(states)):  #len(states) # skip "All States" #do haryana
    safe_click((By.ID, "j_idt43_label"))
    time.sleep(0.4)
    states = driver.find_elements(By.CSS_SELECTOR, state_items_css)

    state = states[i]
    print("\nSTATE:", state.text)
    driver.execute_script("arguments[0].click();", state)
    time.sleep(.5)

    # --- Wait for RTOs ---
    wait.until(lambda d: len(d.find_elements(By.CSS_SELECTOR, rto_items_css)) > 1)
    safe_click((By.ID, "selectedRto_label"))
    time.sleep(0.4)
    rtos = driver.find_elements(By.CSS_SELECTOR, rto_items_css)

    for r in rtos[1:]:   # skip "All Offices"
        print("RTO:", r.text)
        driver.execute_script("arguments[0].click();", r)
        time.sleep(.3)

        # --- Set Y-Axis ---
        safe_click((By.ID, "yaxisVar_label"))
        time.sleep(0.3)
        for opt in driver.find_elements(By.CSS_SELECTOR, yaxis_items_css):
            if "Vehicle Category" in opt.text:
                driver.execute_script("arguments[0].click();", opt)
                break

        # --- Set X-Axis = Month Wise ---
        safe_click((By.ID, "xaxisVar_label"))   # open dropdown
        time.sleep(0.4)

        xaxis_opts = driver.find_elements(By.CSS_SELECTOR, "#xaxisVar_items li")
        for opt in xaxis_opts:
            if "Vehicle Class" in opt.text or "Vehicle Class" in opt.text:
                driver.execute_script("arguments[0].click();", opt)
                break

        # --- Set Year ---
        safe_click((By.ID, "selectedYear_label"))
        time.sleep(0.4)
        for opt in driver.find_elements(By.CSS_SELECTOR, year_items_css):
            if "2025" in opt.text:
                driver.execute_script("arguments[0].click();", opt)
                break

        # --- Click Refresh ---
        safe_click((By.ID, "j_idt85"))
        time.sleep(.5)
        print("\tRefreshed for", state.text, "->", r.text)
        
        
        # --- Select Month ---

        months = ["JAN","FEB","MAR","APR","MAY","JUN","JUL","AUG","SEP","OCT","NOV","DEC"]
        for month in months:
            select_month(month)
            time.sleep(0.3)
            # --- Click Excel download ---
            safe_click((By.ID, "groupingTable:xls"))
            time.sleep(.6)

            # files = sorted([os.path.join(download_dir, f) for f in os.listdir(download_dir)], key=os.path.getmtime)
            # path = files[-1]

            # # --- Detect format ---
            # ext = os.path.splitext(path)[1].lower()

            # try:
            #     if ext in [".xls", ".xlsx"]:
            #         # Try Excel first
            #         try:
            #             df = pd.read_excel(path, engine="openpyxl")  # for .xlsx
            #         except Exception:
            #             # Fallback: treat as CSV if Excel parsing fails
            #             df = pd.read_csv(path)
            #     else:
            #         # Default to CSV
            #         df = pd.read_csv(path)

            #     print("Downloaded shape:", df.shape)


                # --- Clean content ---
                # content = df.iloc[2:, 1:]
                # category_class = df.columns[0]
                # rto_clean = re.sub(r"\s+", "", category_class, flags=re.IGNORECASE)
                # content["RTO"] = [rto_clean] * content.shape[0]
                # content["STATE"] = state.text

                # # --- Check empty/non-empty ---
                # if df.shape[1] > 8 and df.shape[0]>3:
                #     print("✅ Non-empty table for", state.text, "->", r.text)
                # else:
                #     print("⚠️ Empty table for", state.text, "->", r.text)

                # all_dataframes.append(content)
            
            # except Exception as e:
            #     print("❌ Failed to parse downloaded file:", e)

# if all_dataframes:
#     master_df = pd.concat(all_dataframes, ignore_index=True)
#     master_df.to_excel("vahan_master_class.xlsx", index=False)
#     print("Saved consolidated data to vahan_master_class.xlsx")
# else:
#     print("No data collected.")

driver.quit()

In [6]:
if all_dataframes:
    master_df = pd.concat(all_dataframes, ignore_index=True)
    master_df.to_excel("vahan_master_class.xlsx", index=False)
    print("Saved consolidated data to vahan_master.xlsx")

Saved consolidated data to vahan_master.xlsx


In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

driver = webdriver.Chrome()
wait = WebDriverWait(driver, 20)

driver.get("https://vahan.parivahan.gov.in/vahan4dashboard/vahan/vahan/view/reportview.xhtml")

# --- Locators ---
state_label      = (By.ID, "j_idt38_label")
state_items_css  = "#j_idt38_items li"
rto_label        = (By.ID, "selectedRto_label")
rto_items_css    = "#selectedRto_items li"

yaxis_label      = (By.ID, "yaxisVar_label")
yaxis_items_css  = "#yaxisVar_items li"
xaxis_label      = (By.ID, "xaxisVar_label")
xaxis_items_css  = "#xaxisVar_items li"
year_label       = (By.ID, "selectedYear_label")
year_items_css   = "#selectedYear_items li"

refresh_btn      = (By.ID, "j_idt70")

# --- Open States dropdown once to get list ---
wait.until(EC.element_to_be_clickable(state_label)).click()
time.sleep(0.5)
states = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, state_items_css)))

for i in range(1, len(states)):   # skip "All States"
    # Re-open dropdown each loop
    wait.until(EC.element_to_be_clickable(state_label)).click()
    time.sleep(0.4)
    states = driver.find_elements(By.CSS_SELECTOR, state_items_css)

    state = states[i]
    print("\nSTATE:", state.text)
    driver.execute_script("arguments[0].click();", state)
    time.sleep(1.0)

    # --- Wait for RTOs to populate ---
    wait.until(lambda d: len(d.find_elements(By.CSS_SELECTOR, rto_items_css)) > 1)
    rtos = driver.find_elements(By.CSS_SELECTOR, rto_items_css)

    # Open RTO dropdown
    wait.until(EC.element_to_be_clickable(rto_label)).click()
    time.sleep(0.4)
    rtos = driver.find_elements(By.CSS_SELECTOR, rto_items_css)

    for r in rtos[1:]:   # skip "All Offices"
        print("RTO:", r.text)
        driver.execute_script("arguments[0].click();", r)
        time.sleep(1.0)

        # --- Set Y-Axis = Vehicle Category ---
        wait.until(EC.element_to_be_clickable(yaxis_label)).click()
        time.sleep(0.4)
        yaxis_opts = driver.find_elements(By.CSS_SELECTOR, yaxis_items_css)
        for opt in yaxis_opts:
            if "Vehicle Category" in opt.text:
                driver.execute_script("arguments[0].click();", opt)
                break

        # --- Set X-Axis = Month Wise ---
        wait.until(EC.element_to_be_clickable(xaxis_label)).click()
        time.sleep(0.4)
        xaxis_opts = driver.find_elements(By.CSS_SELECTOR, xaxis_items_css)
        for opt in xaxis_opts:
            if "Month Wise" in opt.text:
                driver.execute_script("arguments[0].click();", opt)
                break

        # --- Set Year = 2025 ---
        wait.until(EC.element_to_be_clickable(year_label)).click()
        time.sleep(0.4)
        year_opts = driver.find_elements(By.CSS_SELECTOR, year_items_css)
        for opt in year_opts:
            if "2025" in opt.text:
                driver.execute_script("arguments[0].click();", opt)
                break

        # --- Click Refresh ---
        btn = wait.until(EC.element_to_be_clickable(refresh_btn))
        driver.execute_script("arguments[0].click();", btn)

        # 
        
        time.sleep(3)
        print("   Refreshed for", state.text, "->", r.text)

time.sleep(2)
driver.quit()

In [5]:
%load_ext autoreload
%autoreload 2
import time, base64,ssl, os, re,json5, pprint,random , math, hashlib, inspect,requests
from selenium import webdriver# Controls the browser
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By                   # Locators (ID, CLASS_NAME, XPATH, etc.)
from selenium.webdriver.support.ui import WebDriverWait       # Waits for elements to appear
from selenium.webdriver.support import expected_conditions as EC  # Conditions like "visible", "clickable"
from selenium.webdriver.chrome.options import Options
from urllib.parse import urljoin
import pandas as pd
import numpy as np
from io import StringIO

amfii_path = r"https://www.amfiindia.com/otherdata/fund-performance"

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
from bs4 import BeautifulSoup

STATE_LIMIT = 2

driver = webdriver.Chrome()
wait = WebDriverWait(driver, 25)

driver.get("https://vahan.parivahan.gov.in/vahan4dashboard/vahan/dashboardview.xhtml")

state_label      = (By.ID, "j_idt44_label")
state_items_css  = "#j_idt44_items li"
rto_label        = (By.ID, "selectedRto_label")
rto_items_css    = "#selectedRto_items li"
go_btn           = (By.ID, "j_idt49")
dashboard_panel  = (By.ID, "dashboardContentsPanel")


def apply_sub(text, pattern, replacement='', ignore_case=True):
    if not isinstance(text, str):
        return text
    flags = re.IGNORECASE if ignore_case else 0
    return re.sub(pattern, replacement, text, flags=flags)

def _normalize_whitespace(text: str) -> str:
    if not isinstance(text, str):
        return text
    return re.sub(r"\s+", " ", text).strip()

def _clean_raw_table_html(rawr):
    rawr = apply_sub(rawr, r'<th\b', '<td', ignore_case=True)
    rawr = apply_sub(rawr, r'</th\b', '</td', ignore_case=True)
    
    #tbody
    rawr = re.sub(r"<thead\b",r"<tbody",rawr, re.IGNORECASE)
    rawr = re.sub(r"</thead\b",r"</tbody",rawr, re.IGNORECASE)
    
    #other tags
    rawr = apply_sub(rawr, r"</?(?:strong|sup|b|p|br)(?:\s+[^>]*)?>",ignore_case=True)
    rawr = apply_sub(rawr,r'[*@\n\t]+', ignore_case=True)
    rawr = apply_sub(rawr,r"<tr[^>]*>\s*(?:&nbsp;|\u00A0|\s)*</tr>", ignore_case=True)
    rawr = _normalize_whitespace(rawr)
    
    soup = BeautifulSoup(rawr, "html.parser")
    ALLOWED = {"rowspan", "colspan"}
    for tag in soup.find_all(True):
        for attr in list(tag.attrs):
            if attr not in ALLOWED:
                del tag.attrs[attr]
    final_html = str(soup)
    
    empty = re.findall(r"<table[^>]*>\s*(?:&nbsp;|\u00A0|\s)*</table>", final_html, re.IGNORECASE)
    if empty:
        return ""
    return final_html

# open states
wait.until(EC.element_to_be_clickable(state_label)).click()
time.sleep(1)
states = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, state_items_css)))

for i in range(1, min(1 + STATE_LIMIT, len(states))):

    # re-open / re-fetch each loop
    wait.until(EC.element_to_be_clickable(state_label)).click()
    time.sleep(0.4)
    states = driver.find_elements(By.CSS_SELECTOR, state_items_css)

    state = states[i]
    print("\nSTATE:", state.text)
    driver.execute_script("arguments[0].click();", state)
    time.sleep(1.3)

    # wait for RTOs
    wait.until(lambda d: len(d.find_elements(By.CSS_SELECTOR, rto_items_css)) > 1)
    rtos = driver.find_elements(By.CSS_SELECTOR, rto_items_css)
    print("RTOs:", len(rtos))

    # open rto dropdown
    wait.until(EC.element_to_be_clickable(rto_label)).click()
    time.sleep(0.4)
    rtos = driver.find_elements(By.CSS_SELECTOR, rto_items_css)

    # loop RTOs
    for r in rtos:
        print("RTO:", r.text)
        driver.execute_script("arguments[0].click();", r)
        time.sleep(1.0)

        btn = wait.until(EC.element_to_be_clickable(go_btn))
        driver.execute_script("arguments[0].click();", btn)

        wait.until(EC.presence_of_element_located(dashboard_panel))
        print("      Dashboard Loaded")

        # -------------------------
        # CLICK THE 2025 LINK HERE
        # -------------------------
        try:
            link_2025 = wait.until(
                EC.element_to_be_clickable((By.ID, "j_idt331"))
            )
            print("Clicking 2025 link…")
            driver.execute_script("arguments[0].click();", link_2025)

            wait.until(EC.presence_of_element_located(dashboard_panel))
            print("2025 Loaded Successfully")
            
            nov_data = wait.until(
                EC.element_to_be_clickable((By.ID,"j_idt734"))
            )
            driver.execute_script("arguments[0].click();", nov_data)
            
            element = driver.find_elements(By.CSS_SELECTOR, "panel_vhClass_content table")
            tabl = element.get_attribute("outerHTML")
            final_html = _clean_raw_table_html(tabl)
            df = pd.read_html(StringIO(final_html))[0]
            print(df)

        except Exception as e:
            print("2025 link NOT found for this RTO.")

time.sleep(2)
driver.quit()


TimeoutException: Message: 
Stacktrace:
Symbols not available. Dumping unresolved backtrace:
	0x7ff6eea888e5
	0x7ff6eea88940
	0x7ff6ee86165d
	0x7ff6ee8b9a33
	0x7ff6ee8b9d3c
	0x7ff6ee90df67
	0x7ff6ee90ac97
	0x7ff6ee8aac29
	0x7ff6ee8aba93
	0x7ff6eeda0640
	0x7ff6eed9af80
	0x7ff6eedb96e6
	0x7ff6eeaa5de4
	0x7ff6eeaaed8c
	0x7ff6eea92004
	0x7ff6eea921b5
	0x7ff6eea77ee2
	0x7ffc7518e8d7
	0x7ffc76528d9c


In [ ]:
import requests
import pandas as pd
import time

def fetch_all_cities():
    base_url = "https://www.amfiindia.com/api/distributor-agent"
    throttle = 3
    all_cities = set()

    # First request to get total pages
    params = {
        "strOpt": "ALL",
        "city": "",
        "search": "",
        "page": 1,
        "pageSize": 100
    }

    response = requests.get(base_url, params=params, verify=False)
    response.raise_for_status()
    data = response.json()

    page_count = data["meta"]["pageCount"]
    print(f"Total pages: {page_count}")

    # Collect first page
    for rec in data.get("data", []):
        all_cities.add(rec.get("City"))

    # Loop through remaining pages
    for page in range(2, page_count + 1):
        params["page"] = page
        response = requests.get(base_url, params=params, verify=False)
        response.raise_for_status()
        page_data = response.json()
        for rec in page_data.get("data", []):
            all_cities.add(rec.get("City"))
        time.sleep(throttle)

    return sorted(all_cities)

if __name__ == "__main__":
    cities = fetch_all_cities()
    print(f"Total unique cities: {len(cities)}")
    
    with open("cities.csv", "w", encoding="utf-8") as f:
        for city in cities:
            f.write(city + "\n")
    print(cities)  # show first 50 as sample

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

driver = webdriver.Chrome()
driver.get(amfii_path)
wait = WebDriverWait(driver, 10)

# Switch to iframe
iframe = wait.until(EC.presence_of_element_located((By.TAG_NAME, "iframe")))
driver.switch_to.frame(iframe)

dropdown_1 = ['Open Ended', 'Close Ended']
dropdown_2 = ['Equity', 'Debt', 'Hybrid', 'Solution Oriented', 'Other']
fixed_op4 = "All"

def select_dropdown(dropdown_element, value):
    arrow = dropdown_element.find_element(By.CSS_SELECTOR, ".ng-arrow-wrapper")
    arrow.click()
    time.sleep(0.5)
    options = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, ".ng-option")))
    for opt in options:
        if opt.text.strip() == value:
            opt.click()
            break
    time.sleep(0.5)

for op1 in dropdown_1:
    dropdowns = driver.find_elements(By.CSS_SELECTOR, ".ng-select")
    select_dropdown(dropdowns[0], op1)

    for op2 in dropdown_2:
        dropdowns = driver.find_elements(By.CSS_SELECTOR, ".ng-select")
        select_dropdown(dropdowns[1], op2)

        # Get all op3 options
        dropdowns = driver.find_elements(By.CSS_SELECTOR, ".ng-select")
        arrow = dropdowns[2].find_element(By.CSS_SELECTOR, ".ng-arrow-wrapper")
        arrow.click()
        time.sleep(0.5)
        options = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, ".ng-option")))
        op3_options = [opt.text.strip() for opt in options]
        arrow.click()

        for op3 in op3_options:
            dropdowns = driver.find_elements(By.CSS_SELECTOR, ".ng-select")
            select_dropdown(dropdowns[2], op3)
            select_dropdown(dropdowns[3], fixed_op4)

            # Click "Go" button
            go_button = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button.mat-raised-button")))
            go_button.click()
            # print(f"✅ Selected: {op1} + {op2} + {op3} + {fixed_op4}")
            # time.sleep(5) 
            # Set the date to 30-Sep-2025
            date_input = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "input[formcontrolname='AsOnDate']")))
            date_input.clear()
            date_input.send_keys("30-Sep-2025")
            time.sleep(1)

            # Click the "Download Excel" button
            download_button = wait.until(EC.element_to_be_clickable((By.XPATH, "//button[contains(text(),'Download Excel')]")))
            # download_button.click()
            driver.execute_script("arguments[0].click();", download_button)
            print(f"📥 Download triggered for: {op1} + {op2} + {op3} + {fixed_op4}")
            time.sleep(1)  # wait for download to complete

driver.quit()

In [2]:
dropdown_1 = ['Open Ended', 'Close Ended']
op3_map = {
    "Equity": ['Large Cap', 'Large & Mid Cap', 'Flexi Cap', 'Multi Cap', 'Mid Cap',
               'Small Cap', 'Value', 'ELSS', 'Contra', 'Dividend Yield', 'Focused', 'Sectoral / Thematic'],
    "Debt": ['Long Duration', 'Medium to Long Duration', 'Short Duration', 'Medium Duration',
             'Money Market', 'Low Duration', 'Ultra Short Duration', 'Liquid', 'Overnight',
             'Dynamic Bond', 'Corporate Bond', 'Credit Risk', 'Banking and PSU', 'Floater',
             'FMP', 'Gilt', 'Gilt with 10 year constant duration'],
    "Hybrid": ['Aggressive Hybrid', 'Conservative Hyrbid', 'Equity Savings', 'Arbitrage',
               'Multi Asset Allocation', 'Dynamic Asset Allocation or Balanced Advantage', 'Balanced Hybrid'],
    "Solution Oriented": ["Children's", 'Retirement'],
    "Other": ['Index Funds ETFs', 'FoFs (Overseas/Domestic)']
}

dropdown_4_value = "All"

all_combos = []
for op1 in dropdown_1:
    for op2, op3_list in op3_map.items():
        for op3 in op3_list:
            all_combos.append({
                "op1": op1,
                "op2": op2,
                "op3": op3,
                "op4": dropdown_4_value
            })

# print(f"Total combinations: {len(all_combos)}")
# all_combos
# Small test set
# combos = [
#     {"op1": "Open Ended", "op2": "Debt", "op3": "Overnight", "op4": "All"},
#     {"op1": "Open Ended", "op2": "Debt", "op3": "Liquid", "op4": "All"},
    # {"op1": "Open Ended", "op2": "Equity", "op3": "Large Cap", "op4": "All"},
    # {"op1": "Open Ended", "op2": "Equity", "op3": "Large & Mid Cap", "op4": "All"},
    # {"op1": "Open Ended", "op2": "Equity", "op3": "Flexi Cap", "op4": "All"},
    # {"op1": "Open Ended", "op2": "Equity", "op3": "Multi Cap", "op4": "All"},
    # {"op1": "Open Ended", "op2": "Equity", "op3": "Mid Cap", "op4": "All"},
    # {"op1": "Open Ended", "op2": "Equity", "op3": "Small Cap", "op4": "All"},
    # {"op1": "Open Ended", "op2": "Equity", "op3": "Value", "op4": "All"},
    # {"op1": "Open Ended", "op2": "Equity", "op3": "ELSS", "op4": "All"},
    # {"op1": "Open Ended", "op2": "Equity", "op3": "Contra", "op4": "All"},
    # {"op1": "Open Ended", "op2": "Equity", "op3": "Dividend Yield", "op4": "All"},
# ]

In [ ]:
import os, time, json, shutil, glob
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, StaleElementReferenceException
from selenium.webdriver.common.keys import Keys
import pandas as pd

DATE_STR = "30-Jul-2025"
DOWNLOAD_DIR = os.path.join(os.getcwd(), f"mf_downloads_{DATE_STR}")
os.makedirs(DOWNLOAD_DIR, exist_ok=True)

chrome_options = webdriver.ChromeOptions()
prefs = {
    "download.default_directory": DOWNLOAD_DIR,
    "download.prompt_for_download": False,
    "download.directory_upgrade": True,
    "safebrowsing.enabled": True,
}
chrome_options.add_experimental_option("prefs", prefs)

driver = webdriver.Chrome(options=chrome_options)
wait = WebDriverWait(driver, 10)

driver.get("https://www.amfiindia.com/otherdata/fund-performance")
iframe = wait.until(EC.presence_of_element_located((By.TAG_NAME, "iframe")))
driver.switch_to.frame(iframe)

combos = all_combos

def select_dropdown(index, value, retries=3):
    for attempt in range(retries):
        try:
            # Wait for overlay to disappear if present
            try:
                WebDriverWait(driver, 5).until(
                    EC.invisibility_of_element_located((By.CSS_SELECTOR, ".cdk-overlay-backdrop"))
                )
            except TimeoutException:
                pass  # Continue anyway

            dropdowns = driver.find_elements(By.CSS_SELECTOR, "ng-select")
            dropdown = dropdowns[index]
            driver.execute_script("arguments[0].scrollIntoView(true);", dropdown)

            arrow = dropdown.find_element(By.CSS_SELECTOR, ".ng-arrow-wrapper")
            driver.execute_script("arguments[0].click();", arrow)
            time.sleep(0.5)

            options = WebDriverWait(driver, 5).until(
                EC.presence_of_all_elements_located((By.CSS_SELECTOR, ".ng-option"))
            )

            for opt in options:
                if opt.text.strip() == value:
                    driver.execute_script("arguments[0].scrollIntoView(true);", opt)
                    driver.execute_script("arguments[0].click();", opt)
                    time.sleep(0.5)
                    return

            raise ValueError(f"Option '{value}' not found")
        except Exception as e:
            print(f"Retrying select for '{value}' ({attempt+1}/{retries}) due to {e.__class__.__name__}")
            time.sleep(1)
    raise Exception(f"Failed to select '{value}' after retries")

# def select_dropdown(index, value, retries=3):
#     for attempt in range(retries):
#         try:
#             dropdowns = driver.find_elements(By.CSS_SELECTOR, "ng-select")
#             dropdown = dropdowns[index]
#             arrow = dropdown.find_element(By.CSS_SELECTOR, ".ng-arrow-wrapper")
#             arrow.click()
#             time.sleep(0.5)

#             options = WebDriverWait(driver, 5).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, ".ng-option")))
#             for opt in options:
#                 if opt.text.strip() == value:
#                     driver.execute_script("arguments[0].scrollIntoView(true);", opt)
#                     # opt.click()
#                     driver.execute_script("arguments[0].click();", opt)
#                     time.sleep(0.5)
#                     return
#             raise ValueError(f"Option '{value}' not found")
#         except Exception as e:
#             print(f"Retrying select for '{value}' ({attempt+1}/{retries}) due to {e.__class__.__name__}")
#             time.sleep(1)
#     raise Exception(f"Failed to select '{value}' after retries")

def set_date(date_str):
    date_input = driver.find_element(By.CSS_SELECTOR, "input[bsdatepicker]")
    driver.execute_script("""
        arguments[0].value = arguments[1];
        arguments[0].dispatchEvent(new Event('input', { bubbles: true }));
        arguments[0].dispatchEvent(new Event('change', { bubbles: true }));
    """, date_input, date_str)
    time.sleep(1)

def click_go():
    go_button = driver.find_element(By.CSS_SELECTOR, "button[mat-raised-button]")
    go_button.click()
    time.sleep(2)

def download_excel():
    excel_icon = driver.find_element(By.CSS_SELECTOR, ".excel-border.cursor-pointer")
    excel_icon.click()
    time.sleep(1)

def if_data_avalaible():
    try:
        snackbars = driver.find_elements(By.CSS_SELECTOR, "snack-bar-container")
        for snackbar in snackbars:
            if "Data Not Available" in snackbar.text:
                close_icon = snackbar.find_element(By.XPATH, ".//mat-icon[contains(text(), 'close')]")
                close_icon.click()
                time.sleep(1)
                print("Data Not Available. Snackbar closed.")
                return False
    except:
        pass
    return True


for combo in combos:
    try:
        op1, op2, op3, op4 = combo["op1"], combo["op2"], combo["op3"], combo["op4"]

        print(f"\nPreparing combo: {op1}, {op2}, {op3}, {op4}")
    
        set_date(DATE_STR)
        select_dropdown(0, op1)
        select_dropdown(1, op2)
        select_dropdown(2, op3)
        select_dropdown(3, op4)

        # click_go()
        time.sleep(2)

    except Exception as e:
        print(f"[ERROR] {combo}: {e}")

driver.quit()


In [ ]:
def create_grand_sheet(path:str):
    all_dataframes = []
    excel_files = sorted(glob.glob(os.path.join(path, "*.xlsx")))
    
    all_data_path = os.path.join(path, "ALL_DATA_FILE.xlsx")
    grand_path = os.path.join(path, f"Grand_Stacked_{DATE_STR}.xlsx")
    
    #stack
    for file in excel_files:
        df = pd.read_excel(file, header=None)
        empty_rows = pd.DataFrame([[""] * df.shape[1]] * 3)
        all_dataframes.append(df)
        all_dataframes.append(empty_rows)

    grand_df = pd.concat(all_dataframes, ignore_index=True)
    grand_df.to_excel(grand_path, index=False, header=False)
    
    #one shot
    with pd.ExcelWriter(all_data_path, engine='openpyxl') as writer:
        for i, file in enumerate(excel_files):
            sheet_name = f"Sheet{i+1}"
            df = pd.read_excel(file, header=None)
            df.to_excel(writer, sheet_name=sheet_name, index=False, header=False)

    print(f" `ALL_DATA_FILE` created with {len(excel_files)} sheets: {all_data_path}")
    print(f"Grand Excel created with {len(excel_files)} : {grand_path}")


path = r"C:\Users\kaustubh.keny\Projects\office-work\selenium-scrape\mf_downloads_30-Jul-2025\mod"

create_grand_sheet(path)
